In [1]:
import geopandas as gpd
import os
import warnings
import ast

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
bike_boxes_path = os.path.join(base_dir, "osm_asl.gdb")
unassociated_bike_boxes_path = os.path.join(base_dir, "unassociated_bike_boxes.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)
bike_boxes = gpd.read_file(bike_boxes_path)

# Convert CRS_LEG from string to list
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Reproject bike boxes to match roads CRS
bike_boxes = bike_boxes.to_crs(roads.crs)

# Add 'ID' column to bike boxes and assign sequential values
bike_boxes['ID'] = range(1, len(bike_boxes) + 1)

# Buffer bike boxes by 5 meters
bike_boxes['geometry'] = bike_boxes.geometry.buffer(5)

# Find road segments that intersect with bike boxes
bike_box_roads = roads[roads.geometry.intersects(bike_boxes.unary_union)]
bike_box_road_ids = set(bike_box_roads['unique_id'])

# Create CRS_ASL column and assign 1 where a bike box exists, 0 otherwise
crossings['CRS_ASL'] = crossings['CRS_LEG'].apply(lambda x: 1 if any(seg in bike_box_road_ids for seg in x) else 0)

# Identify unassociated bike boxes (those not linked to any road segment)
unassociated_bike_boxes = bike_boxes[~bike_boxes.intersects(bike_box_roads.unary_union)]

# Save unassociated bike boxes to a new shapefile
unassociated_bike_boxes.to_file(unassociated_bike_boxes_path, driver='ESRI Shapefile')

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

print("Updated crossings.shp with CRS_ASL attribute for bike boxes.")
print(f"There are {len(unassociated_bike_boxes)} unassociated bike boxes.")

Updated crossings.shp with CRS_ASL attribute for bike boxes.
There are 4 unassociated bike boxes.


In [2]:
crossings['CRS_ASL'].value_counts()

CRS_ASL
0    40646
1      157
Name: count, dtype: int64